In [1]:
from qiskit_ibm_provider import IBMProvider
with open('../ibm_API_key','r') as file:
    token = file.readline()

provider = IBMProvider(token=token, instance="ibm-q-ncsu/nc-state/quantum-compiler")

In [2]:
import pandas as pd

data_lines = []
for N in range(4,7):
    for T in range(1,4):
        for noise in range(1,13,2):
            data = {'N' : N, 'T':T,'Noise':noise}
            data_lines.append(data)

df = pd.DataFrame(data_lines)

job_ids = [
    "cna13m6m2pvg008sktx0",
    "cna13nyyzmv0008w8t10",
    "cna13q6m2pvg008sktxg",
    "cna13rfj5tjg0080xg9g",
    "cna13sz1m2c0008wypqg",
    "cna13v7yzmv0008w8t1g",
    "cna151m1m2c0008wypxg",
    "cna152m8ff30008rjhp0",
    "cna153wj5tjg0080xgcg",
    "cna155mr0vyg008d7zpg",
    "cna156wj5tjg0080xgd0",
    "cna158dyzmv0008w8t20",
    "cna15wf1m2c0008wypy0",
    "cna15xzj5tjg0080xgeg",
    "cna15z7j5tjg0080xgf0",
    "cna1600r0vyg008d7zqg",
    "cna16188ff30008rjhsg",
    "cna162g1m2c0008wypyg",
    "cna173mr0vyg008d7zrg",
    "cna174wj5tjg0080xgh0",
    "cna175wr0vyg008d7zs0",
    "cna177c8ff30008rjhwg",
    "cna178x1m2c0008wypzg",
    "cna17an8ff30008rjhx0",
    "cna1944m2pvg008skvag",
    "cna195cr0vyg008d7zx0",
    "cna196m8ff30008rjj1g",
    "cna197w8ff30008rjj20",
    "cna1995r0vyg008d7zxg",
    "cna19anr0vyg008d7zy0",
    "cna19t71m2c0008wyq20",
    "cna19vf1m2c0008wyq2g",
    "cna19wzj5tjg0080xgp0",
    "cna19y7j5tjg0080xgpg",
    "cna19zfyzmv0008w8t6g",
    "cna1a0r1m2c0008wyq30",
    "cna1ahaj5tjg0080xgqg",
    "cna1ajaj5tjg0080xgr0",
    "cna1akjm2pvg008skvd0",
    "cna1an2yzmv0008w8t80",
    "cna1ap28ff30008rjj4g",
    "cna1apt8ff30008rjj5g",
    "cna1b34r0vyg008d8010",
    "cna1b4c1m2c0008wyq4g",
    "cna1b5w1m2c0008wyq5g",
    "cna1b74r0vyg008d801g",
    "cna1b85j5tjg0080xgsg",
    "cna1b9dr0vyg008d8020",
    "cna1br7m2pvg008skvh0",
    "cna1bsf1m2c0008wyq80",
    "cna1btq8ff30008rjja0",
    "cna1bvz1m2c0008wyq90",
    "cna1bwz8ff30008rjjag",
    "cna1bxz8ff30008rjjb0"
]

df['Job Id'] = job_ids

In [3]:
print(df)

    N  T  Noise                Job Id
0   4  1      1  cna13m6m2pvg008sktx0
1   4  1      3  cna13nyyzmv0008w8t10
2   4  1      5  cna13q6m2pvg008sktxg
3   4  1      7  cna13rfj5tjg0080xg9g
4   4  1      9  cna13sz1m2c0008wypqg
5   4  1     11  cna13v7yzmv0008w8t1g
6   4  2      1  cna151m1m2c0008wypxg
7   4  2      3  cna152m8ff30008rjhp0
8   4  2      5  cna153wj5tjg0080xgcg
9   4  2      7  cna155mr0vyg008d7zpg
10  4  2      9  cna156wj5tjg0080xgd0
11  4  2     11  cna158dyzmv0008w8t20
12  4  3      1  cna15wf1m2c0008wypy0
13  4  3      3  cna15xzj5tjg0080xgeg
14  4  3      5  cna15z7j5tjg0080xgf0
15  4  3      7  cna1600r0vyg008d7zqg
16  4  3      9  cna16188ff30008rjhsg
17  4  3     11  cna162g1m2c0008wypyg
18  5  1      1  cna173mr0vyg008d7zrg
19  5  1      3  cna174wj5tjg0080xgh0
20  5  1      5  cna175wr0vyg008d7zs0
21  5  1      7  cna177c8ff30008rjhwg
22  5  1      9  cna178x1m2c0008wypzg
23  5  1     11  cna17an8ff30008rjhx0
24  5  2      1  cna1944m2pvg008skvag
25  5  2    

In [4]:
df['raw_result'] = df['Job Id'].apply(lambda x: provider.retrieve_job(x).result().get_counts())

In [5]:
def get_average_distributions(same_circuit_results):

    all_keys = set()
    for dist in same_circuit_results:
        all_keys = all_keys.union(set(dist.keys()))

    from collections import defaultdict

    def default_value():
        return 0

    average_dist = defaultdict(default_value)

    for key in all_keys:
        for i in range(len(same_circuit_results)):
            if key in same_circuit_results[i]:
                average_dist[key] =  average_dist[key] + same_circuit_results[i][key]

        average_dist[key] = average_dist[key] // len(same_circuit_results)

    return average_dist

df['average_result'] = df['raw_result'].apply(lambda x: get_average_distributions(x))

In [6]:
def process_results(result,layout):

    def layout_rev(res):
        n = len(layout)
        # print(self.layout)
        b = res
        ret = ""
        for i in range(n):
            ret += b[-1 - layout[i]]
        return ret


    n_shots = sum(result.values())

    from collections import defaultdict

    def default_value():
        return 0

    ret = defaultdict(default_value)

    for key in result.keys():
        new_key = layout_rev(key)
        ret[new_key] += result[key] / n_shots

    return ret

def get_layout(row):
    layout = None
    if row['N'] == 4:
        layout = [0,1,2,3]
    elif row['N'] == 5:
        layout = [0,1,2,3,5]
    else:
        layout = [0,1,2,3,5,8]

    return process_results(row['average_result'],layout)

df['processed_result'] = df.apply(lambda x: get_layout(x), axis=1)

In [7]:
#Getting the ideal results
from SimuQ.thesis.Experiment.utilities.hamiltonian_models import Ising
from simuq.qutip import QuTiPProvider
import numpy as np

def get_ideal_result(N,T):
    #Ising chain model creation
    h = np.array([1 for j in range(N)])
    J_chain = np.zeros((N, N))
    for j in range(N - 1):
        J_chain[j, j + 1] = 1

    J_cycle = np.copy(J_chain)
    J_cycle[0, N - 1] = 1
    Ising_chain = Ising(N, T, J_chain, h)

    #Classical simulation
    qtpp = QuTiPProvider()
    qtpp.compile(Ising_chain)
    qtpp.run()

    return qtpp.results()

df['ideal_result'] = df.apply(lambda x: get_ideal_result(x['N'],x['T']), axis=1)

Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.
Compiled.
Solved.


In [8]:
from qiskit.quantum_info import hellinger_fidelity
from SimuQ.thesis.Experiment.utilities.evaluation_metrics import TV

df['Hellinger fidelity'] = df.apply(lambda x: hellinger_fidelity(x['ideal_result'],x['processed_result']), axis=1)
df['Total Variational Distance'] = df.apply(lambda x: TV(x['ideal_result'],x['processed_result']), axis=1)

In [9]:
print(df[['N','T','Noise','Hellinger fidelity','Total Variational Distance']])

    N  T  Noise  Hellinger fidelity  Total Variational Distance
0   4  1      1            0.928722                    0.186743
1   4  1      3            0.769959                    0.351347
2   4  1      5            0.776780                    0.375950
3   4  1      7            0.695552                    0.439344
4   4  1      9            0.675728                    0.442812
5   4  1     11            0.764835                    0.342631
6   4  2      1            0.860347                    0.291313
7   4  2      3            0.736569                    0.419337
8   4  2      5            0.725219                    0.422130
9   4  2      7            0.700404                    0.457199
10  4  2      9            0.689274                    0.470114
11  4  2     11            0.692479                    0.465068
12  4  3      1            0.663586                    0.496030
13  4  3      3            0.655878                    0.518363
14  4  3      5            0.664281     

In [10]:
df['Top 10 ideal dist'] = df['ideal_result'].apply(lambda x: dict(sorted(x.items(), key= lambda y: y[1], reverse=True)[:10]))
print(df.columns)

Index(['N', 'T', 'Noise', 'Job Id', 'raw_result', 'average_result',
       'processed_result', 'ideal_result', 'Hellinger fidelity',
       'Total Variational Distance', 'Top 10 ideal dist'],
      dtype='object')


In [11]:
def filter_dist(row):
    ideal_dist_keys = list(row['Top 10 ideal dist'].keys())
    dist = row['processed_result']

    top_10_expected_dist = {}
    for key in ideal_dist_keys:
        top_10_expected_dist[key] = dist[key]

    return top_10_expected_dist

df['Top 10 expected dist'] = df.apply(lambda x: filter_dist(x),axis=1)

In [45]:
zne_dfs = []

for N in range(4,7):
    for T in range(1,4):
        ideal_dist = df[(df['N'] == N) & (df['T'] == T) & (df['Noise'] == 1)]['Top 10 ideal dist'].iat[0]
        keys = list(ideal_dist.keys())

        def append_noise_data(row):
            new_dict = {'Noise' : row['Noise']}
            new_dict.update(row['Top 10 expected dist'])
            row['Top 10 expected dist'] = new_dict
            return row

        ideal_data = {'Noise' : 0}
        ideal_data.update(ideal_dist)
        zne_df_data = [ideal_data]

        zne_df_data = zne_df_data + list(df[(df['N'] == N) & (df['T'] == T)][['Noise','Top 10 expected dist']].apply(lambda x: append_noise_data(x), axis=1)['Top 10 expected dist'])
        zne_df = pd.DataFrame(zne_df_data)

        zne_dfs.append(zne_df)

number_of_points = 4
df_id = 0
degree = 1

def normalize(distA, distB):
    total_A_sum = 1 - sum(distA.values())
    total_B_sum = 0

    for key in distA:
        total_B_sum += distB[key]

    total_B_sum = 1 - total_B_sum

    distC = {}
    for key in distB:
        if key in distA:
            distC[key] = distA[key]
        else:
            distC[key] = distB[key] * total_A_sum / total_B_sum

    return distC

def extrapolate(col,x_points):
    if col.name == 'Noise':
        temp = pd.Series(['ZNE mitigated probability'])
    else:
        y = list(col)
        coefficients = np.polyfit(x_points, y, degree)
        poly_function = np.poly1d(coefficients)
        predicted_y_value = 0 if poly_function(0) < 0 else poly_function(0)     #Need to confirm with Professor
        temp = pd.Series([predicted_y_value])
    return temp

zne_mitigated_df_list = []

for N in range(4,7):
    for T in range(1,4):

        x_points = np.array([i for i in range(1,number_of_points * 2,2)])

        y_df = zne_dfs[df_id][1:number_of_points + 1]
        y_df_mitigated = y_df.apply(lambda x: extrapolate(x,x_points), axis=0)

        zne_dfs[df_id] = pd.concat([zne_dfs[df_id],y_df_mitigated],axis=0,ignore_index=True)
        record = y_df_mitigated.to_dict(orient='records')[0]

        record.pop('Noise',None)

        ideal_dist = df[(df['N'] == N) & (df['T'] == T) & (df['Noise'] == 1)]['ideal_result'].iat[0]
        dist = normalize(record,ideal_dist)

        zne_mitigated_data = {
            'N': N,
            'T':T,
            'Mitigated Distribution': dist,
            'Ideal Distribution' : ideal_dist,
            'Hellinger fidelity' : hellinger_fidelity(dist, ideal_dist),
            'Initial Hellinger fidelity' : df[(df['N'] == N) & (df['T'] == T) & (df['Noise'] == 1)]['Hellinger fidelity'].iat[0],
            'Total variational distance' : TV(dist,ideal_dist),
            'Initial Total Variational distance' : df[(df['N'] == N) & (df['T'] == T) & (df['Noise'] == 1)]['Total Variational Distance'].iat[0]}

        zne_mitigated_df_list.append(zne_mitigated_data)
        df_id += 1

zne_mitigated_df = pd.DataFrame(zne_mitigated_df_list)

with pd.ExcelWriter('top_10_probabilities.xlsx',engine='xlsxwriter') as writer:
    count = 0
    for N in range(4,7):
        for T in range(1,4):
            zne_dfs[count].to_excel(writer,sheet_name='N_{}_T_{}'.format(N,T),index=False)
            count += 1

In [46]:
zne_mitigated_df

,N,T,Mitigated Distribution,Ideal Distribution,Hellinger fidelity,Initial Hellinger fidelity,Total variational distance,Initial Total Variational distance
0,4,1,"{'0000': 0.1778245674619108, '0001': 0.0520480...","{'0000': 0.21072942962058566, '0001': 0.109919...",0.966814,0.928722,0.149512,0.186743
1,4,2,"{'0000': 0.08705373070311322, '0001': 0.065093...","{'0000': 0.2770214655429199, '0001': 0.0598548...",0.841930,0.860347,0.305844,0.291313
2,4,3,"{'0000': 0.047423864372826086, '0001': 0.07761...","{'0000': 0.24648402578991985, '0001': 0.018513...",0.651050,0.663586,0.519089,0.496030
3,5,1,"{'00000': 0.06441866318119739, '00001': 0.0890...","{'00000': 0.16859825082132646, '00001': 0.0980...",0.932765,0.900929,0.212680,0.247188
4,5,2,"{'00000': 0.07331711873592966, '00001': 0.0283...","{'00000': 0.03419323155440778, '00001': 0.0796...",0.910306,0.820804,0.278310,0.315448
5,5,3,"{'00000': 0.008177090601991129, '00001': 0.010...","{'00000': 0.39208989435406016, '00001': 0.0142...",0.231704,0.196954,0.825781,0.824929
6,6,1,"{'000000': 0.0718545645400407, '000001': 0.037...","{'000000': 0.13909131418319828, '000001': 0.07...",0.939279,0.871577,0.220197,0.274199
7,6,2,"{'000000': 0.013408353432121104, '000001': 0.0...","{'000000': 0.058408399848135006, '000001': 0.0...",0.884796,0.724343,0.308069,0.426466
8,6,3,"{'000000': 0.016317222853242504, '000001': 0.0...","{'000000': 0.2216031757421513, '000001': 0.028...",0.734320,0.608091,0.462087,0.508490


In [47]:
temp_dfs = []

for N in range(4,7):
    for T in range(1,4):
        dist = zne_mitigated_df[(zne_mitigated_df['N'] == N) & (zne_mitigated_df['T'] == T)]['Mitigated Distribution'].iat[0]
        ideal_dist = zne_mitigated_df[(zne_mitigated_df['N'] == N) & (zne_mitigated_df['T'] == T)]['Ideal Distribution'].iat[0]

        temp_df = pd.DataFrame([dist,ideal_dist])
        temp_dfs.append(temp_df)

with pd.ExcelWriter('zne_results.xlsx',engine='xlsxwriter') as writer:
    count = 0
    zne_mitigated_df.to_excel(writer,sheet_name='zne_results',index=False)
    for N in range(4,7):
        for T in range(1,4):
            temp_dfs[count].to_excel(writer,sheet_name='N_{}_T_{}'.format(N,T),index=False)
            count += 1